In [1]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_ChanRed, CapsEEGNet
import torch
import os

In [4]:
model = EEGNet(nb_classes = 2, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  50946


In [3]:
model_SEED = EEGNet_ChanRed(nb_classes = 3, Chans = 62, InnerChans=14, Samples = 200, dropoutRate = 0.1, kernLength = 25, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model_SEED.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  53045


In [4]:
model_capseegnet = CapsEEGNet(nb_classes=3)

total_params = sum(p.numel() for p in model_capseegnet.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  1218320


In [5]:
x = torch.randn(8, 1, 62, 200)
out = model_capseegnet(x)
print(out.shape)

torch.Size([8, 3, 16, 1])


/Users/juliengadonneix/anaconda3/envs/intern3A/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712608633180/work/aten/src/ATen/native/Convolution.cpp:1032.)
  return F.conv2d(input, weight, bias, self.stride,


In [6]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

8
0
mps
False


In [7]:
import subprocess
import plistlib

def get_gpu_info():
    """Get information about the GPU on macOS."""
    # Run system_profiler command and capture the output
    result = subprocess.run(['system_profiler', '-xml', 'SPDisplaysDataType'], capture_output=True)
    if result.returncode == 0:
        # Parse the XML output
        display_info = plistlib.loads(result.stdout)
        # Extract GPU information
        gpu_count = len(display_info[0]['_items'])
        return gpu_count
    else:
        print("Error:", result.stderr)
        return None

gpu_count = get_gpu_info()
if gpu_count is not None:
    print("Number of GPUs:", gpu_count)


Number of GPUs: 1


In [8]:
import time
import torch

start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.330582857131958


In [9]:
t = torch.randn(8,8)
print(t)
torch.reshape(t, (-1, 2, 8))

tensor([[-0.8214, -0.5754,  0.3624,  0.6802,  1.1466, -0.0746,  1.6991, -0.3078],
        [-0.1754,  0.0233, -0.3428, -0.8156,  1.0823, -1.9877,  1.0358, -1.1099],
        [ 0.9743, -1.3674,  0.0821, -1.5815,  0.5267, -0.6714,  0.7230,  0.8020],
        [ 1.1423,  0.3159, -0.4820, -0.6300, -0.6149,  0.1760,  0.7113,  0.2717],
        [ 0.3112,  1.1855,  2.3711,  0.8102,  0.3052,  0.7419, -0.1312,  1.3173],
        [ 1.0911,  0.9252,  0.6750, -0.7153, -0.1869,  1.1169,  0.3641, -0.7933],
        [-0.4154, -0.5017, -1.4023,  0.5756,  1.0989, -0.3077, -0.7895, -1.8963],
        [ 0.2416, -0.9497, -0.3430, -0.2214, -1.0370,  0.7675,  0.9197,  1.0277]])


tensor([[[-0.8214, -0.5754,  0.3624,  0.6802,  1.1466, -0.0746,  1.6991,
          -0.3078],
         [-0.1754,  0.0233, -0.3428, -0.8156,  1.0823, -1.9877,  1.0358,
          -1.1099]],

        [[ 0.9743, -1.3674,  0.0821, -1.5815,  0.5267, -0.6714,  0.7230,
           0.8020],
         [ 1.1423,  0.3159, -0.4820, -0.6300, -0.6149,  0.1760,  0.7113,
           0.2717]],

        [[ 0.3112,  1.1855,  2.3711,  0.8102,  0.3052,  0.7419, -0.1312,
           1.3173],
         [ 1.0911,  0.9252,  0.6750, -0.7153, -0.1869,  1.1169,  0.3641,
          -0.7933]],

        [[-0.4154, -0.5017, -1.4023,  0.5756,  1.0989, -0.3077, -0.7895,
          -1.8963],
         [ 0.2416, -0.9497, -0.3430, -0.2214, -1.0370,  0.7675,  0.9197,
           1.0277]]])